In [51]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib as mt
from dateutil.relativedelta import *

np.set_printoptions(suppress=True)

In [52]:
def compute_daily_returns(df):
    daily_returns=(df/df.shift(1))-1
    df=df.fillna(value=0)
    daily_returns=daily_returns[1:]
    return daily_returns


In [53]:
def var_calculator(data_frame, start_date, end_date):
    value_at_risk_matrix = []
    returns_daily = compute_daily_returns(data_frame)
    
    for symbol in kse30_symbols:
        returns_matrix = returns_daily.loc[start_date : end_date,'{}'.format(symbol)]
        return_matrix = np.array(returns_matrix)
        value_at_risk = np.percentile(return_matrix, 100 * (1-0.99))
        #print(value_at_risk)
        cvarcalc = np.nanmean(returns_matrix < value_at_risk)
        #print(cvarcalc)
        value_at_risk_matrix.append(cvarcalc)
    var_df = pd.DataFrame(data = value_at_risk_matrix, index=kse30_symbols)
    return var_df

In [54]:
def stock_picker(data_frame, start_date, start_date_test, end_date_test,number_of_stocks_selected,principal_amount,principal_amount_all):
    var_df = var_calculator(data_frame, start_date, start_date_test)
    var_df.sort_values(by = 0 ,axis = 0, ascending = False, inplace = True)
    
    ######### Calculating for the best combination ########################
    amount_in_one = principal_amount / number_of_stocks_selected
    
    symbols_to_invest = var_df.index.values.tolist()[0:number_of_stocks_selected]
    symbols_to_invest_df = data_frame.loc[start_date_test:end_date_test ,symbols_to_invest]
    symbols_to_invest_returns = compute_daily_returns(symbols_to_invest_df)
    symbols_to_invest_returns = symbols_to_invest_returns + 1
    symbols_to_invest_returns.iloc[0,:] = symbols_to_invest_returns.iloc[0,:] * amount_in_one  
    symbols_to_invest_returns = symbols_to_invest_returns.cumprod()
    symbols_to_invest_returns['Aggregated_Return_Combination'] = symbols_to_invest_returns.iloc[:, 0:].sum(axis=1)
    symbols_to_invest_returns = symbols_to_invest_returns.iloc[:,number_of_stocks_selected:]
    
    ########## Calculating for all stocks ##########################
    amount_in_all = principal_amount_all / len(data_frame.columns)
    
    symbols_all = var_df.index.values.tolist()[0:len(data_frame.columns)]
    symbols_all_df = data_frame.loc[start_date_test:end_date_test ,symbols_all]
    symbols_all_returns = compute_daily_returns(symbols_all_df)
    symbols_all_returns = symbols_all_returns + 1
    symbols_all_returns.iloc[0,:] = symbols_all_returns.iloc[0,:] * amount_in_all
    symbols_all_returns = symbols_all_returns.cumprod()
    symbols_all_returns['Aggregated_Return_all']= symbols_all_returns.iloc[:, 0:].sum(axis=1)
    symbols_all_returns = symbols_all_returns.iloc[:,len(data_frame.columns):]
    
    ########## Merging Both and returning ##########################
    df = symbols_all_returns.merge(symbols_to_invest_returns,how='outer', left_index=True, right_index=True)
    
    return df

In [77]:
def back_tester(data_frame,start_date,no_of_training_months,number_of_stocks_selected,principal_amount,principal_amount_all):
    start_date = pd.to_datetime(start_date)
    start_date_test =  start_date+relativedelta(months=+no_of_training_months)
    #start_date_test =  start_date+relativedelta(days=+1)
    end_date_test = start_date_test+relativedelta(months=+1)
    funds_returns = stock_picker(df,start_date,start_date_test,end_date_test,number_of_stocks_selected,principal_amount,principal_amount_all)
    updated_amount = funds_returns.iloc[funds_returns.shape[0]-1:,0:]
    #updated_amount_all = funds_returns.iloc[funds_returns.shape[0]-1:,0:]
    return updated_amount['Aggregated_Return_Combination'].iloc[0],updated_amount['Aggregated_Return_all'].iloc[0],funds_returns

In [78]:
start_date = '2018-01-01'
end_date = '2018-12-07'
dates = pd.date_range(start_date, end_date)
df = pd.DataFrame(index = dates) 
df_temp = pd.read_csv('E://Stock_Calculation//data.csv', index_col = 'Date', parse_dates = True)
#df = df.iloc[:,1:]
#df.iloc[:,0] = pd.to_datetime(df.iloc[:,0])
#df = df.set_index('Date')
df = df.join(df_temp)
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

In [79]:
kse30_symbols = list(df)

In [80]:
max_date = df.index.max()
min_date = df.index.min()
no_of_months_in_data = (max_date.year - min_date.year) * 12 + max_date.month - min_date.month
principal_amount = 100000
principal_amount_all = 100000
Start_month = 7

In [81]:
Total_Returns = []
for x in range(Start_month,no_of_months_in_data+1):
    updated_amount,updated_amount_all, returns = back_tester(df,min_date,x,3,principal_amount,principal_amount_all)
    principal_amount = updated_amount
    principal_amount_all = updated_amount_all
    #if x == no_of_months_in_data:
    Total_Returns.append(returns)
    del returns,updated_amount,updated_amount_all

C:\ProgramData\Anaconda3\lib\site-packages\numpy\lib\function_base.py:3652: RuntimeWarning: Invalid value encountered in percentile
  interpolation=interpolation)


In [82]:
Total_Returns

[            Aggregated_Return_all  Aggregated_Return_Combination
 2018-08-02           99870.983075                   99822.117033
 2018-08-03           99985.446020                   99877.455871
 2018-08-04           99985.446020                   99877.455871
 2018-08-05           99985.446020                   99877.455871
 2018-08-06          100131.669568                  100038.829949
 2018-08-07          100164.869564                  100011.100668
 2018-08-08          100253.606043                  100006.895183
 2018-08-09          100408.848330                  100136.710150
 2018-08-10          100483.109352                  100137.926455
 2018-08-11          100483.109352                  100137.926455
 2018-08-12          100483.109352                  100137.926455
 2018-08-13          100518.264378                  100090.662774
 2018-08-14          100585.525762                  100102.148579
 2018-08-15          100587.547406                  100042.078443
 2018-08-1

In [34]:
start_date = '2016-01-01'
end_date = '2018-10-31'
dates = pd.date_range(start_date, end_date)
df = pd.DataFrame(index = dates) 
df_temp = pd.read_csv('E://Stock_Calculation//KSE_30.csv', index_col = 'Date', parse_dates = True)
df = df.join(df_temp)
df = df.fillna(method='ffill')
df = df.fillna(method='bfill')

In [35]:
kse30_symbols = list(df)

In [48]:
max_date = df.index.max()
min_date = df.index.min()
no_of_months_in_data = (max_date.year - min_date.year) * 12 + max_date.month - min_date.month
principal_amount = 100000
Start_month = 30

In [49]:
Total_Returns = []
for x in range(Start_month,no_of_months_in_data+1):
    updated_amount, returns = back_tester(df,min_date,x,3,principal_amount)
    principal_amount = updated_amount
    #if x == no_of_months_in_data:
    Total_Returns.append(returns)
    del returns,updated_amount

In [50]:
Total_Returns

[            Aggregated_Return_all  Aggregated_Return_Combination
 2018-07-02           99518.359719                   98760.675640
 2018-07-03           99029.446319                   97980.027149
 2018-07-04           95586.404333                   94997.304624
 2018-07-05           95625.549596                   94830.478678
 2018-07-06           95948.364456                   94567.344211
 2018-07-07           95948.364456                   94567.344211
 2018-07-08           95948.364456                   94567.344211
 2018-07-09           92799.478457                   93295.401289
 2018-07-10           93143.996920                   94067.046050
 2018-07-11           93405.455468                   94435.319161
 2018-07-12           94205.494281                   94080.001865
 2018-07-13           94861.064368                   93575.761198
 2018-07-14           94861.064368                   93575.761198
 2018-07-15           94861.064368                   93575.761198
 2018-07-1